## Importing packages and attributes

In [1]:
import pandas as pd
import re
from geopy.distance import great_circle

## Data Processing

In this notebook we will process the raw data scraped in our *`TripAdvisor_scraper.ipynb`* We start by loading the CSV file, we created.

In [2]:
trip_df = pd.read_csv("Tripadvisordata_raw.csv")
trip_df.tail() # prints to check if the read went well

,God pris,Mad,Restaurant,Service,Stemning,Location,Number of reviews,Price class,Main rating,Ranking on list,Price range,Type of food,Address,Type of food link
2315,NaN,NaN,Cafe Juicy,NaN,NaN,"55.67448,12.54951",545.0,NaN,NaN,NaN,NaN,NaN,"Gammel Kongevej 85, Frederiksberg, København 1...",NaN
2316,NaN,NaN,Tap 21 Craft Beer,NaN,NaN,"55.67835,12.54699",186.0,--$,NaN,NaN,NaN,Bar,"H. C. Ørsteds Vej 21, Frederiksberg, København...","<div class=""header_links""><a href=""/Restaurant..."
2317,NaN,NaN,Green Room Restaurant &amp; Bar,NaN,NaN,"55.67973,12.53345",12.0,--$$-$$$---------,NaN,NaN,262,Europæisk,"Falkoner Alle 9, Frederiksberg, København 2000...","<div class=""header_links""><a href=""/Restaurant..."
2318,NaN,NaN,Pauseriet,NaN,NaN,"55.67427,12.555178",1120.0,--$$-$$$---,NaN,NaN,121,Café,"Vodroffsvej 2A, Frederiksberg, København 1900 ...","<div class=""header_links""><a href=""/Restaurant..."
2319,NaN,NaN,Pizza 13,NaN,NaN,"55.68324,12.5345",12.0,--$$-$$$---,NaN,NaN,148,Pizza,"Rolfsvej 13, Frederiksberg, København 2000 Dan...","<div class=""header_links""><a href=""/Restaurant..."


### Restructuring data

In [3]:
trip_df = trip_df.replace(regex=['&amp;'], value='&')
trip_df['Main rating'] = trip_df['Main rating'].replace(regex=[','], value='.')
trip_df['Good price'] = trip_df['God pris'] / 10
trip_df['Food'] = trip_df['Mad'] / 10
trip_df['Service'] = trip_df['Service'] / 10
trip_df['Atmosphere'] = trip_df['Stemning'] / 10
trip_df = trip_df[['Restaurant', 'Main rating', 'Ranking on list', 'Price range', 'Price class', 'Location', 'Good price', 'Food', 'Service', 'Atmosphere', 'Type of food', 'Number of reviews', 'Address', 'Type of food link']]

In [4]:
trip_df.head()

,Restaurant,Main rating,Ranking on list,Price range,Price class,Location,Good price,Food,Service,Atmosphere,Type of food,Number of reviews,Address,Type of food link
0,Restaurant Grønnegade,4.5,28.0,664,--$$$$---,"55.681705,12.583386",4.5,4.5,4.5,4.5,Dansk,526.0,"Grønnegade 39, København 1107 Danmark","<div class=""header_links""><a href=""/Restaurant..."
1,Basso København,4.5,18.0,496,--$$-$$$---------,"55.68388,12.58657",4.5,4.5,4.5,NaN,Europæisk,345.0,"Dronningens Tvaergade 22, København 1302 Danmark","<div class=""header_links""><a href=""/Restaurant..."
2,Enomania,4.5,1.0,NaN,--$$-$$$------,"55.670773,12.531963",4.5,5.0,5.0,4.5,Europæisk,245.0,"Vesterbrogade 187, Frederiksberg, København 18...","<div class=""header_links""><a href=""/Restaurant..."
3,Restaurant Krebsegaarden,5.0,2.0,396,--$$$$---,"55.67872,12.569877",4.5,5.0,5.0,4.5,Dansk,1311.0,"Studiestraede 17, København 1455 Danmark","<div class=""header_links""><a href=""/Restaurant..."
4,The Pescatarian,5.0,6.0,597,--$$$$------,"55.68799,12.596316",4.5,5.0,5.0,NaN,Dansk,201.0,Amaliegade 49 On the corner of Amaliegade & Es...,"<div class=""header_links""><a href=""/Restaurant..."


In [5]:
# edit missing data
trip_df["Price class"] = trip_df["Price class"].replace(np.nan, '--$$$$$$', regex=True)

# Translate the price class, to a numeric value

string = trip_df["Price class"]
string = [word.replace('nan','--$$$$$$') for word in string]

# find indexnumbers for more places with missing with missing data
list = []
for i in range(len(string)):    
    if "$" not in string[i]:
        list.append(i)

for i in list: 
    string[i] = '--$$$$$$'

In [6]:
# Filter the "-"'s      
new = []
for i in string:
    x = i.split('--')[1]
    new.append(x)


trip_df["New price class"] = new

#the tranlating keys
dollarsign = ["$", "$-$$", "$$", "$$-$$$", "$$$", "$$$-$$$$","$$$$", "$$$$-$$$$$", "$$$$$", '$$$$$$']
values = [1, 1, 1, 2, 2, 2, 3, 3, 3, 'null']

# make dataframe
dict_ = {'New price class': ["$", "$-$$", "$$", "$$-$$$", "$$$", "$$$-$$$$","$$$$", "$$$$-$$$$$", "$$$$$", '$$$$$$'], 'Price class numeric': [1, 1, 1, 2, 2, 2, 3, 3, 3, 99]}
translate_priceclass = pd.DataFrame(dict_)

new_trip_df = pd.merge(trip_df, translate_priceclass, on='New price class', how='left')

In [7]:
# Make our own ranking system
new_trip_df["Reverse ranking on list"] = (-1)*new_trip_df["Ranking on list"]

# sort on rating, ranking on list and number of reviews
new_trip_df = new_trip_df.sort_values(by=["Main rating", "Reverse ranking on list", "Number of reviews"])

# Add ranking
new_trip_df["Full ranking"] = range(1,len(new_trip_df["Number of reviews"])+1)

In [8]:
#køkken_list = ['Café', 'Afrikansk','Amerikansk','Arabisk','Argentinsk','Armensk','Aserbajdsjansk','Asiatisk','Bar','Belgisk','Brasiliansk','Britisk','Cajun og kreolsk','Cambodjansk','Canadisk','Caribisk','Centralasiatisk','Centraleuropæisk','Centralitaliensk','Dansk','Delikatesseforretning','Egyptisk','Etiopisk','Europæisk','Fastfood','Filippinsk','Fisk og skaldyr','Fra Lazio','Fra Shanghai','Fransk','Fusion','Gademad','Gastropub','Grill','Grillmad','Græsk','Hawaiiansk','Hollandsk','Hongkong','Indiansk','Indisk','Indonesisk','International','Irsk','Israelsk','Italiensk','Japansk','Kantonesisk','Kinesisk','Koreansk','Kroatisk','Latinamerikansk','Libanesisk','Malaysisk','Marokkansk','Mellemamerikansk','Mellemøstlig','Mexicansk','Middelhavsområdet','Moderne','Mongolsk','Nepalesisk','New Zealand','Norditaliensk','Norsk','Pakistansk','Persisk','Peruviansk','Pizza','Portugisisk''Pub','Russisk','Schweizisk','Siciliansk','Singaporeansk','Skandinavisk','Spansk','Specialiteter fra Beijing','Spisested','Steakhouse','Sund','Supper','Sushi','Svensk','Sydamerikansk','Syditaliensk','Szechuan','Taiwansk','Thai','Tibetansk','Toscansk','Tyrkisk','Tysk','Venezuelansk','Vietnamesisk','Vinstue','Xinjiang','Ølpub','Østeuropæisk','Østrigsk']
#most_pop = ['Café', "Italiensk", "Pizza", "Indisk", "Fisk og skaldyr", "Thai", "Steakhouse", "Mexikansk"]
#
#link_list = new_trip_df["Type of food link"]
#link_df = pd.DataFrame(link_list)
#
#food_list_1 = [] #list of list over kitchens
#for link in food_list_1:
#    food_list_1.append([x for x in køkken_list if x in link])
#    
#food_list_2 = []
#for link in food_list_1:
#    food_list_2.append([x for x in most_pop if x in link])
#food_list_2
#
#new_trip_df["Pop kitchen"] = food_list_1

In [14]:
køkken_list = ['Café', 'Afrikansk','Amerikansk','Arabisk','Argentinsk','Armensk','Aserbajdsjansk','Asiatisk','Bar','Belgisk','Brasiliansk','Britisk','Cajun og kreolsk','Cambodjansk','Canadisk','Caribisk','Centralasiatisk','Centraleuropæisk','Centralitaliensk','Dansk','Delikatesseforretning','Egyptisk','Etiopisk','Europæisk','Fastfood','Filippinsk','Fisk og skaldyr','Fra Lazio','Fra Shanghai','Fransk','Fusion','Gademad','Gastropub','Grill','Grillmad','Græsk','Hawaiiansk','Hollandsk','Hongkong','Indiansk','Indisk','Indonesisk','International','Irsk','Israelsk','Italiensk','Japansk','Kantonesisk','Kinesisk','Koreansk','Kroatisk','Latinamerikansk','Libanesisk','Malaysisk','Marokkansk','Mellemamerikansk','Mellemøstlig','Mexicansk','Middelhavsområdet','Moderne','Mongolsk','Nepalesisk','New Zealand','Norditaliensk','Norsk','Pakistansk','Persisk','Peruviansk','Pizza','Portugisisk''Pub','Russisk','Schweizisk','Siciliansk','Singaporeansk','Skandinavisk','Spansk','Specialiteter fra Beijing','Spisested','Steakhouse','Sund','Supper','Sushi','Svensk','Sydamerikansk','Syditaliensk','Szechuan','Taiwansk','Thai','Tibetansk','Toscansk','Tyrkisk','Tysk','Venezuelansk','Vietnamesisk','Vinstue','Xinjiang','Ølpub','Østeuropæisk','Østrigsk']
most_pop = ['Café', "Italiensk", "Pizza", "Indisk", "Fisk og skaldyr", "Thai", "Steakhouse", "Mexikansk"] # gathered from TripAdvisor

link_list = new_trip_df["Type of food link"]
link_df = pd.DataFrame(link_list)

list_test = [] #list of list over kitchens
for link in link_list:
    list_test.append([x for x in køkken_list if x in link])
    
link_test2 = []
for link in link_list:
    link_test2.append([x for x in most_pop if x in link])
link_test2

new_trip_df["Pop kitchen"] = link_test2

TypeError: argument of type 'float' is not iterable

Calculatin distance to our measure point.

In [8]:
distance_list = []
Kgs_Nytorv = '55.679977,12.5841893' #longitude and latitude for Kongens Nytorv

#calculating distance from nytorv to the coordinates in the list
def distance(x):
    Start = new_trip_df["Location"][x]
    Stop = Kgs_Nytorv
    distance_list.append(great_circle(Start, Stop).meters)
    
for x in trip_df.index:
    distance(x)
    
#appending to df 
new_trip_df["Distance from Kgs. Nytorv (m)"] = distance_list 
new_trip_df['Distance from Kgs. Nytorv (m)'] = new_trip_df['Distance from Kgs. Nytorv (m)'].round()

In [18]:
new_trip_df['Postal code'] = [str(x).split('København')[-1].split('Danmark')[0] for x in new_trip_df['Address']]

In [19]:
#trip_df = trip_df.sort_values(by='Ranking on list', ascending=True)
#new_trip_df.to_csv("Tripadvisordata_final.csv", index=False)
new_trip_df.head()

,Restaurant,Main rating,Ranking on list,Price range,Price class,Location,Good price,Food,Service,Atmosphere,Type of food,Number of reviews,Address,Type of food link,New price class,Price class numeric,Reverse ranking on list,Full ranking,Distance from Kgs. Nytorv (m),Postal code
2142,Burger King,1.0,2148.0,NaN,--$$$$$$,"55.65107,12.50931",NaN,NaN,NaN,NaN,NaN,2.0,"Ellebjergvej 142, København 2450 Danmark",NaN,$$$$$$,99,-2148.0,1,199.0,2450
2147,Almanac,1.0,2146.0,NaN,---,"55.67788,12.591933",NaN,NaN,NaN,NaN,Dansk,2.0,"Havnegade 44, København Danmark","<div class=""header_links""><a href=""/Restaurant...",$$$$$$,99,-2146.0,2,459.0,
2130,Star Midnight Kebab-Grill,1.0,2143.0,101,--$$-$$$---,"55.6679,12.54941",NaN,NaN,NaN,NaN,Grill,3.0,"Istedgade 101, København Danmark","<div class=""header_links""><a href=""/Restaurant...",$$-$$$,2,-2143.0,3,3431.0,
2134,Sunset Boulevard,1.0,2137.0,NaN,--$$$$$$,"55.67502,12.580593",NaN,NaN,NaN,NaN,NaN,2.0,"Københavns Hovedbanegård, København 1570 Danmark",NaN,$$$$$$,99,-2137.0,4,908.0,1570
2096,Dwaraka Indisk Restaurant,1.0,2104.0,141,--$$-$$$---,"55.72732,12.524776",NaN,NaN,NaN,NaN,Indisk,1.0,"Frederiksborgvej 221, St, København 2860 Danmark","<div class=""header_links""><a href=""/Restaurant...",$$-$$$,2,-2104.0,5,1171.0,2860
